In [1]:
import logging
import sys
import torch
from llama_index.core import PromptTemplate, Settings, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0, 1"
# 定义日志
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/langchain/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [6]:
# 定义system prompt
SYSTEM_PROMPT = """你是一个人工智能助手， 请根据检索到的上下文回答问题。"""
query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

# 使用llama-index创建本地大模型
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name='./model_local/qwen/Qwen1.5-14B-Chat',
    model_name='./model_local/qwen/Qwen1.5-14B-Chat',
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16},
)
Settings.llm = llm

# 使用llama-index-embeddings-huggingface构建本地embedding模型
Settings.embed_model = HuggingFaceEmbedding(
    model_name="./embedding_model"
)


Loading checkpoint shards: 100%|██████████| 8/8 [00:20<00:00,  2.51s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from llama_index.core import SimpleDirectoryReader
# 读取文档
documents = SimpleDirectoryReader("./ops_txt_temp").load_data()

In [4]:
# 对文档进行切分，将切分后的片段转化为embedding向量，构建向量索引
index = VectorStoreIndex.from_documents(documents, 
                                        transformations=[SentenceSplitter(chunk_size=256)])
# 将embedding向量和向量索引存储到文件中
index.storage_context.persist(persist_dir='./data_base/vector_db/index_store_ops')


In [7]:
# 构建查询引擎
query_engine = index.as_query_engine(similarity_top_k=5)
# 查询获得答案
response = query_engine.query("PCF与NRF对接时，一般需要配置哪些数据？")
print(response)

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/root/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


为了使PCF能够与NRF正确对接并进行服务注册和发现，一般需要配置以下数据：

1. NRF的地址和端口号：PCF需要知道NRF的具体连接地址和通信端口，以便进行服务注册和通过NRF发现其他NF实例。

2. PCF支持的号段信息：PCF需要提供它所支持的服务号码范围，这有助于其他NF通过NRF进行匹配选择。

3. PCF允许的网络切片列表：如果PCF需要对特定的网络切片进行限制或支持，那么需要配置允许的网络切片信息，以便其他NF在正确的切片范围内发现PCF。

4. PCF支持的SNSSAI列表：SNSSAI是用于标识网络切片的，PCF需要配置它支持的SNSSAI列表，以便其他NF通过SNSSAI进行匹配。

5. 可能的优先级、权重或动态负荷策略：如果存在多个满足条件的PCF，AMF可能需要基于这些策略来选择一个PCF。

6. 普通升级后拨测号段配置或服务号段配置：在不同场景下，PCF可能从不同的配置中获取号段信息。

配置完成后，PCF会向NRF注册服务，同时也会订阅NRF提供的其他服务，如CHF、UDR、AMF、SMF、BSF等，以获取用户相关的信息和业务需求。


In [8]:
from llama_index.core.node_parser import SimpleNodeParser

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)

# Extract nodes from documents
nodes = node_parser.get_nodes_from_documents(documents)

In [11]:
# This block of code is for educational purposes 
# to showcase what the nodes looks like
i=0

print(f"Text: \n{nodes[1].text}")

Text: 
处理系统告警

维护目的

对系统的告警进行检查，包括当前告警和历史告警，按照告警的级别，及时排查告警原因，确保系统的正常运行。

要按告警严重程度的次序及时处理，对于原因不明的告警要及时分析，随时掌握系统的异常情况。

维护指导

待处理告警

  1. 使用admin账号登录TECS Director管理门户网站。

  2. 将鼠标移动至上方的按钮处，出现菜单栏页面。单击菜单栏页面中的告警管理→当前告警→告警监控按钮，打开告警监控页面。

  3. 单击页面中告警级别排序，使待处理告警按照告警级别由高到低排序。

  4. 单击每条告警的告警码名称查看告警详情，并按照处理意见进行处理。




历史告警

  1. 使用admin账号登录TECS Director管理门户网站。

  2. 将鼠标移动至上方的按钮处，出现菜单栏页面。单击菜单栏页面中的告警管理→历史告警按钮，打开告警查询页面，默认显示最近一天新增的历史告警。

  3. 检查最近一天新增的历史告警，并单击每条告警的操作栏中的按钮查看告警详情，分析告警产生的原因。




参考标准

  * 查询出的当前告警记录为空，或已确认告警原因。

  * 历史告警均已明确原因，并处理完成。




异常处理

发现系统有任何当前告警或未处理的历史告警都应及时分析并尽快解决。告警处理的方法参见告警处理。如果仍无法消除告警，及时联系中兴通讯技术支持。


In [12]:
from llama_index.core import VectorStoreIndex, StorageContext
# from llama_index.vector_stores.weaviate import WeaviateVectorStore
# 从存储文件中读取embedding向量和向量索引

storage_context = StorageContext.from_defaults(persist_dir='./data_base/vector_db/index_store_ops')

index = VectorStoreIndex(
    nodes,
    storage_context = storage_context,
)

In [13]:
# 构建查询引擎
query_engine = index.as_query_engine(similarity_top_k=5)
# 查询获得答案
response = query_engine.query("PCF与NRF对接时，一般需要配置哪些数据？")
print(response)

为了使PCF与NRF成功对接，一般需要配置以下数据：

1. **NRF注册失败时是否停止周期性注册**：这通常是一个开关，可以选择“是”或“否”，默认可能是“否”。

2. **令牌有效期允许最大偏差(秒)**：用于PCF验证入局请求令牌的有效期，允许的最大偏差范围，例如默认设置为10秒或60秒。

3. **大包消息缓存最大个数**：可能影响NRF处理大流量数据的能力。

4. **区域限制用于服务发现**：可能涉及到服务的地理定位和访问控制。

5. **注册管理携带负荷**：可能指NRF如何处理注册请求的负载均衡。

6. **注册超时等待时长(秒)**：PCF等待NRF响应的超时时间。

7. **NRF修改注册信息**：是否允许NRF动态更新其注册信息。

请根据实际情况调整这些配置。


In [15]:
retriever = index.as_retriever(similarity_top_k=5)
res = retriever.retrieve("PCF与NRF对接时，一般需要配置哪些数据？")
for i in res:
    print(i)

Node ID: 9b9e71fd-55d7-46b6-8069-14d4d7861c5d
Text: # ADD PCFSERVICE   ## 命令功能  该命令用于增加PCF支持的服务信息。  当需要PCF对外提供相关服务时，
使用该命令。设置成功后，PCF会向NRF注册该服务的FQDN、优先级和版本信息，供后续其他NF发现该PCF服务进行正常业务处理。  ##
注意事项  需要根据实际规划要求填写地址、端口和版本等信息。如果配置错误，PCF无法向NRF注册服务。
HTTP服务端模板ID需要在“服务端模板配置（ADD SERVERPROFILE）”中配置。  ## 输入参数说明  标识 | 名称 |
类型 | 说明   ---|---|---|---   SERVICETYPE | 服务类型 | 参数可选性:必选参数；参数类型:枚举。
参见枚举定义。 | 参数作用：本参数标识PCF...
Score:  0.676

Node ID: fe02aea4-9b85-458f-af09-0545b112012a
Text: # 服务号段配置  ## 背景知识  PCF在向NRF注册时，会携带pcfinfo信息，该信息描述了PCF支持的号段。  ##
功能描述  此配置用于配置PCF支持的号段信息。  PCF支持的号段信息是PCF的基础配置，PCF向NRF注册时会携带号段信息，其他NF在通
过NRF发现PCF时，可以通过号段对PCF进行匹配选择。
该配置为可选配置，如果不配置，则其他NF通过NRF发现PCF时就无法通过号段对PCF进行筛选。
在拨测流程中，PCF从普通升级后拨测号段配置（查询普通升级后拨测号段））中获取号段信息，在非拨测流程中，PCF从服务号段配置中获取号段信息。
## 相关主题    * **ADD SERVNUM**      * **SET SERVNUM**      * **D...
Score:  0.676

Node ID: 31a7dd3b-2ca6-4f73-b598-a5d8e72fcdf9
Text: 默认值:0。 | 参数作用：本参数指定该服务支持的容量，用于对端在多个相同PCF服务中，根据每个PCF配置的容量，计算本服务在所
有的PCF服务中所占权重，对端根据权重选择相应的PCF服务，对端可以根据每个PCF的服务能力进行负